# Data Challenge - Πρόβλεψη Πληρότητας Πτήσεων

Στα πλαίσια της εργασίας του μαθήματος "Εξόρυξη Γνώσης από Βάσεις Δεδομένων και τον Παγκόσμιο Ιστό", θα δουλέψετε πάνω σε ένα πρόβλημα κατηγοριοποίησης. Συγκεκριμένα, σας δίνεται ένα σύνολο δεδομένων το οποίο αποτελείται από μερικές χιλιάδες πτήσεις, όπου κάθε πτήση περιγράφεται απο ένα σύνολο μεταβλητών (αεροδρόμιο αναχώρησης, αεροδρόμιο άφιξης, κτλ). Κάθε πτήση χαρακτηρίζεται επίσης από μια μεταβλητή που σχετίζεται με τον αριθμό των επιβατών της πτήσης (π.χ. κάθε τιμή της μεταβλητής σχετίζεται με ενα εύρος πλήθους επιβατών). Για κάποιες πτήσεις, η τιμή της μεταβλητής  είναι γνωστή, ενώ για άλλες όχι. Στόχος σας είναι να προβλέψετε την τιμή της μεταβλητής για τις πτήσεις για τις οποίες δεν είναι διαθέσιμη.

### Σύνολο Δεδομένων

Το αρχείο με όνομα `train.csv` περιέχει τα δεδομένα εκπαίδευσης (training set) του προβλήματος, ενώ το αρχείο `test.csv` περιέχει τα δεδομένα ελέγχου (test set) του προβλήματος. Κάθε γραμμή των δυο αυτών αρχείων αντιστοιχεί σε μια πτήση η οποία χαρακτηρίζεται από τις εξής μεταβλητές:

Μεταβλητή | Περιγραφή
--- | --- 
DateOfDeparture | Ημερομηνία αναχώρησης
Departure | Κωδικός αεροδρομίου αναχώρησης
CityDeparture | Πόλη αναχώρησης
LongitudeDeparture 	 | Γεωγραφικό μήκος αεροδρομίου αναχώρησης
LatitudeDeparture 	 | Γεωγραφικό πλάτος αεροδρομίου αναχώρησης
Arrival | Κωδικός αεροδρομίου άφιξης
CityArrival | Πόλη άφιξης
LongitudeArrival | Γεωγραφικό μήκος αεροδρομίου άφιξης
LatitudeArrival | Γεωγραφικό πλάτος αεροδρομίου άφιξης
WeeksToDeparture | Πόσες εβδομάδες πριν την αναχώρηση της πτήσης κατά μέσο όρο έκλεισαν οι επιβάτες τα εισητήριά τους
std_wtd | Τυπική απόκλιση για το παραπάνω 
Το training set περιέχει μια επιπλέον μεταβλητή (`PAX`) η οποία έχει σχέση με τον αριθμό των επιβατών της πτήσης. Η μεταβλητή αυτή παίρνει 8 διαφορετικές τιμές (τιμές από 0 έως 7 οπότε 8 κατηγορίες συνολικά). Κάθε κατηγορία υποδηλώνει πόσοι περίπου επιβάτες χρησιμοποίησαν την πτήση. Οι αριθμοί στις κατηγορίες έχουν ανατεθεί με τυχαίο τρόπο, οπότε μην θεωρήσετε ότι η κατηγορία 0 υποδηλώνει πολύ λίγους επιβάτες ενώ η κατηγορία 7 πάρα πολλούς επιβάτες. Η μεταβλητή `PAX` λείπει από το test set καθώς πρόκειται για την μεταβλητή που πρέπει να προβλέψετε στα πλαίσια της παρούσας εργασίας.

Παρακάτω σας δίνεται κώδικας ο οποίος φορτώνει τα δεδομένα εκπαίδευσης σε ένα DataFrame της βιβλιοθήκης Pandas και τυπώνει τις πρώτες 5 γραμμές. Οπότε μπορείτε να δείτε τις 12 μεταβλητές του προβλήματος.

In [1]:
import pandas as pd

df_train = pd.read_csv('fTrain.csv')
df_train.head()

,DateOfDeparture,Departure,CityDeparture,LongitudeDeparture,LatitudeDeparture,Arrival,CityArrival,LongitudeArrival,LatitudeArrival,WeeksToDeparture,...,CV,CV of norm,Distance,Norm Distance,Airports,Day,Month,Year,IsHolidays,PAX
0,2011-12-05,ORD,Chicago,0.747393,0.671019,EWR,Newark,0.688004,0.938417,8.352941,...,0.678473,0.137959,1527.939887,0.246995,2,5,12,2011,0,7
1,2013-03-01,ATL,Atlanta,0.362189,0.738700,LGA,New York,0.691918,0.944181,10.421053,...,0.959764,0.418331,1181.336943,0.184571,1,1,3,2013,0,7
2,2012-02-06,ORD,Chicago,0.747393,0.671019,BOS,Boston,0.765205,1.000000,9.250000,...,0.771548,0.230730,1879.752387,0.310356,2,6,2,2012,0,7
3,2012-12-10,ORD,Chicago,0.747393,0.671019,SFO,San Francisco,0.546078,0.000000,8.666667,...,0.854341,0.313252,3833.459735,0.662220,2,10,12,2012,0,7
4,2012-04-13,SFO,San Francisco,0.546078,0.000000,JFK,New York,0.685569,0.946004,14.037037,...,0.702324,0.161732,5403.554833,0.944995,2,13,4,2012,0,7


Επίσης, όσον αφορά τις διαστάσεις του training set, όπως βλέπετε παρακάτω, αποτελείται από 8899 γραμμές και 12 στήλες.

In [2]:
df_train.shape

(8899, 23)

Δηλαδή έχετε διαθέσιμα τα δεδομένα απο 8899 πτήσεις για τις οποίες ξέρετε την κατηγορία του αριθμού επιβάτών `PAX`.

Αντίθετα, το αρχείο με όνομα `test.csv` περιέχει το test set του προβλήματος. Όπως και στην περίπτωση του training set, σας δίνεται και τώρα κώδικας ο οποίος φορτώνει το test set σε ένα DataFrame της βιβλιοθήκης Pandas και τυπώνει τις πρώτες 5 γραμμές του.

In [3]:
df_test = pd.read_csv('fTest.csv')
df_test.head()

,DateOfDeparture,Departure,CityDeparture,LongitudeDeparture,LatitudeDeparture,Arrival,CityArrival,LongitudeArrival,LatitudeArrival,WeeksToDeparture,...,Norm std,CV,CV of norm,Distance,Norm Distance,Airports,Day,Month,Year,IsHolidays
0,2012-10-21,DFW,Dallas-Fort Worth,0.328023,0.493226,SFO,San Francisco,0.546078,0.000000,14.600000,...,0.632963,0.792866,0.178700,2821.532925,0.479971,2,21,10,2012,0
1,2012-09-13,LAX,Los Angeles,0.376310,0.077221,ATL,Atlanta,0.362189,0.738700,14.730769,...,0.753192,0.907237,0.268260,3779.583120,0.652517,2,13,9,2012,0
2,2012-09-04,ORD,Chicago,0.747393,0.671019,IAH,Houston,0.193537,0.526253,8.470588,...,0.250433,0.694822,0.101925,823.505267,0.120126,2,4,9,2012,1
3,2012-08-13,DEN,Denver,0.649638,0.344594,PHX,Phoenix,0.352841,0.201740,8.200000,...,0.277814,0.767421,0.158775,845.501646,0.124087,2,13,8,2012,0
4,2012-09-10,ORD,Chicago,0.747393,0.671019,SEA,Seattle,1.000000,0.001277,12.090909,...,0.469124,0.755829,0.149698,3825.798893,0.660840,2,10,9,2012,0


Όσον αφορά τις διαστάσεις του test set, όπως βλέπετε παρακάτω, αποτελείται από 2229 γραμμές και 11 στήλες. Η στήλη που λείπει αφορά τη μεταβλητή `PAX` την οποία στόχος σας είναι να προβλέψετε.

In [4]:
df_test.shape

(2229, 22)

### Στόχος

Όπως αναφέρθηκε και παραπάνω, στόχος σας είναι να προβλέψετε την κατηγορία αριθμού επιβατών (μεταβλητή `PAX`) για κάθε πτήση του test set. Πρόκειται για ένα supervised learning πρόβλημα. Θα πρέπει να επεξεργαστείτε τα δεδομένα του training set και στη συνέχεια, θα τα χρησιμοποιήσετε για να εκπαιδεύσετε κάποιο ταξινομητή, τον οποίο θα χρησιμοποιήσετε για να κάνετε προβλέψεις σχετικά με τα δεδομένα του test set.


### Αξιολόγηση

Η αξιολόγηση των προβλέψεων θα γίνει με βάση το `micro F1-score`. Το F1-score παίρνει τιμές μεταξύ 0 και 1. Όσο μεγαλύτερη η τιμή του F1-score, τόσο καλύτερος ο ταξινομητής που παρήγαγε τις προβλέψεις. Για κάθε μια από τις 8 κατηγορίες του test set μας, ο ταξινομητής θα κάνει κάποιες σωστές προβλέψεις και κάποιες λάθος. Πιο συγκεκριμένα, για μια κατηγορία $c$:
- τα παραδείγματα που ανήκουν στην κατηγορία $c$ και ο ταξινομητής προέβλεψε ότι πράγματι ανήκουν στην κατηγορία $c$ ονομάζονται True Positives (TP).
- τα παραδείγματα που ανήκουν στην κατηγορία $c$ και ο ταξινομητής προέβλεψε λανθασμένως ότι ανήκουν σε κάποια άλλη κατηγορία ονομάζονται False Positives (FP).
- τα παραδείγματα που ανήκουν σε κάποια κατηγορία διαφορετική της $c$ και ο ταξινομητής προέβλεψε ότι πράγματι ανήκουν σε κάποια κατηγορία διαφορετική της $c$ ονομάζονται True Negatives (TN).
- τα παραδείγματα που ανήκουν σε κατηγορία διαφορετική της $c$ και ο ταξινομητής προέβλεψε ότι ανήκουν στην κατηγορία $c$ ονομάζονται False Negatives (FN).

Δεδομένων των παραπάνω ορισμών, μπορούμε συνήθως να υπολογίσουμε μερικές μετρικές απόδοσης του ταξινομητή μας, λαμβάνοντας υπόψη τον συνολικό (για όλες τις κατηγορίες) αριθμό True Positives (TP), False Positives (FP) και False Negatives (FN).
Στο πρόβλημα που σας έχει ανατεθεί υπάρχουν συνολικά 8 κατηγορίες $c_1, c_2, \ldots, c_8$ οπότε μπορούμε να υπολογίσουμε:
Πρώτα, το `Recall` το οποίο ορίζεται ως:
$$ Recall = \frac{\sum_{i=1}^8 TP_{c_i}}{\sum_{i=1}^8TP_{c_i}+\sum_{i=1}^8FN_{c_i}} $$
και μετράει συνολικά το ποσοστό των παραδειγμάτων που προβλέφτηκαν ότι ανήκουν στην κατηγορία $c_i$ σε σχέση με αυτά που ανήκουν πράγματι στην κατηγορία $c_i$. Έπειτα, το `Precision` το οποίο μας λέει πόσα από τα παραδείγματα που προβλέφτηκαν ότι ανήκουν στην κατηγορία $c_i$, ανήκουν πράγματι σε αυτή την κατηγορία, και ορίζεται ως εξης:
$$ Precision = \frac{\sum_{i=1}^8 TP_{c_i}}{\sum_{i=1}^8TP_{c_i}+\sum_{i=1}^8FP_{c_i}} $$

Το `micro F1-score` ειναι ο αρμονικός μέσος του Precision και του Recall, και ορίζεται ως εξής:
$$ micro\text{ } F1-score =  \frac{2*Recall*Precision}{Recall + Precision} $$
Πρόκειται για μια ευρέως διαδεδομένη μετρική απόδοσης η οποία είναι και αξιόπιστη δεδομένου ότι λαμβάνει υπόψη τόσο τα False Positives όσο και τα False Negatives.

Αφότου υποβάλετε μια λύση στην πλατφόρμα (όπως περιγράφεται παρακάτω), το micro F1-score της λύσης σας υπολογίζεται αυτόματα και εμφανίζεται στην οθόνη.

### Υποβολή Λύσης

Αφότου έχετε σχεδιάσει και τρέξει τον αλγοριθμό σας, και έχετε προβλέψει την κατηγορία επιβατών για κάθε πτήση του test set, πρέπει να υποβάλετε την λύση σας στην πλατφόρμα Kaggle ώστε να αξιολογηθεί. Συγκεκριμένα, στην κεντρική οθόνη υπάρχει η επιλογή `Submit Predictions` η οποία σας δίνει τη δυνατότητα να ανεβάσετε τις προβλέψεις σας. Οι προβλέψεις σας θα πρέπει να συμπεριληφθούν σε ένα αρχείο όπου κάθε γραμμή περιέχει την πρόβλεψή σας για την πτήση η οποία βρίσκεται στην ίδια γραμμή του test set. Για παράδειγμα, παρακάτω σας δίνονται οι τρείς πρώτες πτήσεις του test set:
``` html
2012-10-21,DFW,Dallas-Fort Worth,32.896828,-97.037997,SFO,San Francisco,37.618972,-122.374889,14.6,11.5758369028
2012-09-13,LAX,Los Angeles,33.942536,-118.408075,ATL,Atlanta,33.636719,-84.428067,14.7307692308,13.3643037748
2012-09-04,ORD,Chicago,41.978603,-87.904842,IAH,Houston,29.984433,-95.341442,8.47058823529,5.88555060146
```
Η πρώτη γραμμή του αρχείου που θα υποβάλετε θα πρέπει να περιέχει την πρόβλεψή σας για την κατηγορία αριθμού επιβατών της πρώτης πτήσης, η δεύτερη γραμμή θα πρέπει να περιέχει την πρόβλεψή σας για την δεύτερη πτήση και αντίστοιχα η τρίτη γραμμή θα πρέπει να περιέχει την πρόβλεψή σας για την τρίτη πτήση.

**Προσοχή**: Στο αρχείο που θα υποβάλετε, κάθε γραμμή θα πρέπει να περιέχει 2 αριθμούς, έναν για το `Id` της πτήσης του test set και έναν για το `Label` που υποδηλώνει την κατηγορία επιβατών (αριθμοί 0-7). Επιπλέον, θα πρέπει να υπάρχουν συνολικά **2229** γραμμές όσες και οι πτήσεις για τις οποίες πρέπει να κάνετε πρόβλεψη. Τέλος, η κωδικοποίηση του αρχείου θα πρέπει να είναι **utf-8**. Φυσικά θα υπάρχει και μία ακόμη γραμμή στην αρχή που περιέχει τό όνομα των πεδίων `Id` και `Label`.

### Αρχικός Κώδικας Python

Σας δίνεται ένας αρχικός Python κώδικας ο οποίος χρησιμοποιεί ως μόνη πληροφορία το αεροδρόμιο από το οποίο αναχώρησε η πτήση και το αεροδρόμιο στο οποίο προσγειώθηκε για να προβλέψει την κατηγορία αριθμού επιβατών. Συγκεκριμένα, ο παρακάτω κώδικας αποθηκεύει στη μεταβλητή `y_train` τη στήλη `PAX` του training set, δηλαδή την στήλη την οποία θέλουμε να μάθουμε να προβλέπουμε. Στη συνέχεια, διαγράφει όλες τις στήλες του training set εκτός από τις `Departure` και `Arrival` οι οποίες αντιστοιχούν στους κωδικούς αεροδρομίων αναχώρησης και άφιξης. Επίσης για να είναι πιό εύκολος ο έλεγχος της λύσης σας (τοπικά), μπορείτε αν θέλετε να παράγετε ένα νέο training και test set απο το αρχικό training set, χρησιμοποιώντας την  `train_test_split` του `scikit-learn` (σχολιασμένη γραμμή παρακάτω).

In [5]:
df_train.head()

,DateOfDeparture,Departure,CityDeparture,LongitudeDeparture,LatitudeDeparture,Arrival,CityArrival,LongitudeArrival,LatitudeArrival,WeeksToDeparture,...,CV,CV of norm,Distance,Norm Distance,Airports,Day,Month,Year,IsHolidays,PAX
0,2011-12-05,ORD,Chicago,0.747393,0.671019,EWR,Newark,0.688004,0.938417,8.352941,...,0.678473,0.137959,1527.939887,0.246995,2,5,12,2011,0,7
1,2013-03-01,ATL,Atlanta,0.362189,0.738700,LGA,New York,0.691918,0.944181,10.421053,...,0.959764,0.418331,1181.336943,0.184571,1,1,3,2013,0,7
2,2012-02-06,ORD,Chicago,0.747393,0.671019,BOS,Boston,0.765205,1.000000,9.250000,...,0.771548,0.230730,1879.752387,0.310356,2,6,2,2012,0,7
3,2012-12-10,ORD,Chicago,0.747393,0.671019,SFO,San Francisco,0.546078,0.000000,8.666667,...,0.854341,0.313252,3833.459735,0.662220,2,10,12,2012,0,7
4,2012-04-13,SFO,San Francisco,0.546078,0.000000,JFK,New York,0.685569,0.946004,14.037037,...,0.702324,0.161732,5403.554833,0.944995,2,13,4,2012,0,7


In [6]:
y_train = df_train[['PAX']]

from sklearn.model_selection import train_test_split
#df_train, df_test, y_train, y_test = train_test_split(df_train, y_train, test_size=0.2, random_state=42)

df_train.drop(df_train.columns[[1,5,10,11,12,13,14,15,16,17,18,19,20,21,22]], axis=1, inplace=True) #best keep 0,2,6,9,distance

df_train.head()

,DateOfDeparture,CityDeparture,LongitudeDeparture,LatitudeDeparture,CityArrival,LongitudeArrival,LatitudeArrival,WeeksToDeparture
0,2011-12-05,Chicago,0.747393,0.671019,Newark,0.688004,0.938417,8.352941
1,2013-03-01,Atlanta,0.362189,0.738700,New York,0.691918,0.944181,10.421053
2,2012-02-06,Chicago,0.747393,0.671019,Boston,0.765205,1.000000,9.250000
3,2012-12-10,Chicago,0.747393,0.671019,San Francisco,0.546078,0.000000,8.666667
4,2012-04-13,San Francisco,0.546078,0.000000,New York,0.685569,0.946004,14.037037


Για να μπορεί να δουλέψει ένα αλγόριθμος ταξινόμησης είναι απαραίτητο το training και το test set να έχουν ακριβώς τον ίδιο αριθμό στηλών (ίδιες μεταβλητές). Συνεπώς, πρέπει και στο test set να διαγράψουμε όλες τις στήλες εκτός από τις Departure και Arrival. Αυτό γίνεται με τον παρακάτω κώδικα.

In [7]:
df_test.drop(df_test.columns[[1,5,10,11,12,13,14,15,16,17,18,19,20,21]], axis=1, inplace=True)

df_test.head()

,DateOfDeparture,CityDeparture,LongitudeDeparture,LatitudeDeparture,CityArrival,LongitudeArrival,LatitudeArrival,WeeksToDeparture
0,2012-10-21,Dallas-Fort Worth,0.328023,0.493226,San Francisco,0.546078,0.000000,14.600000
1,2012-09-13,Los Angeles,0.376310,0.077221,Atlanta,0.362189,0.738700,14.730769
2,2012-09-04,Chicago,0.747393,0.671019,Houston,0.193537,0.526253,8.470588
3,2012-08-13,Denver,0.649638,0.344594,Phoenix,0.352841,0.201740,8.200000
4,2012-09-10,Chicago,0.747393,0.671019,Seattle,1.000000,0.001277,12.090909


Οι στήλες των training και test set περιέχουν κατηγορικές μεταβλητές των οποίων οι τιμές είναι αλφαριθμητικά. Οι αλγόριθμοι ταξινόμησης ωστόσο δουλεύουν μόνο με αριθμητικές τιμές. Χρησιμοποιούμε το αντικείμενο `LabelEncoder` του `scikit-learn` για να μετατρέψουμε τα αλφαριθμητικά σε αριθμητικές τιμές.

In [8]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
#le.fit(df_train['Departure'])
#df_train['Departure'] = le.transform(df_train['Departure'])
#df_train['Arrival'] = le.transform(df_train['Arrival'])
#df_test['Departure'] = le.transform(df_test['Departure'])
#df_test['Arrival'] = le.transform(df_test['Arrival'])
le.fit(df_train['CityDeparture'])
df_train['CityDeparture'] = le.transform(df_train['CityDeparture'])
df_train['CityArrival'] = le.transform(df_train['CityArrival'])
df_test['CityDeparture'] = le.transform(df_test['CityDeparture'])
df_test['CityArrival'] = le.transform(df_test['CityArrival'])
le = LabelEncoder()
le.fit(df_train['DateOfDeparture'])
df_train['DateOfDeparture'] = le.transform(df_train['DateOfDeparture'])
df_test['DateOfDeparture'] = le.transform(df_test['DateOfDeparture'])

In [9]:
#PCA
'''
from sklearn.decomposition import PCA
samples = df_train.shape[0]
dim = df_train.shape[1]
n_components=3
pca = PCA(n_components)
Xtr_reduced = pca.fit_transform(df_train)
Xte_reduced = pca.fit_transform(df_test)
'''

'\nfrom sklearn.decomposition import PCA\nsamples = df_train.shape[0]\ndim = df_train.shape[1]\nn_components=3\npca = PCA(n_components)\nXtr_reduced = pca.fit_transform(df_train)\nXte_reduced = pca.fit_transform(df_test)\n'

Έπειτα, εκπαιδεύουμε έναν ταξινομητή logistic regression για να προβλέψουμε τις κατηγορίες αριθμού επιβατών των δεδομένων ελέγχου. Επιπλέον, αποθηκεύουμε τις προβλέψεις μας στο αρχείο `y_pred.txt` στο δίσκο.

In [10]:
from sklearn.linear_model import LogisticRegression
import numpy as np

X_train = df_train
X_test = df_test
y_train = np.ravel(y_train)

clf = LogisticRegression()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

X_train.shape

C:\Users\IAKOVOS\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\IAKOVOS\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


(8899, 8)

In [11]:
import csv
with open('y_predict.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['Id', 'Label'])
    for i in range(y_pred.shape[0]):
        writer.writerow([i, y_pred[i]])

In [12]:
from sklearn.metrics import f1_score
#y_test = np.loadtxt('test_labels.csv', delimiter=",", skiprows=1, usecols=[1]) # test_labels.csv το χουν αυτοι? 
#f1_score(y_test, y_pred, average='micro')

Υποβάλλουμε το αρχείο `y_pred.csv` στην πλατφόρμα και μας δίνει micro F1-score ίσο με 0.23.

Ένας εναλλακτικός τρόπος αναπαράστασης κατηγορικών μεταβλητών είναι το λεγόμενο one-hot encoding όπου υπάρχει διαθέσιμη μια μεταβλητή για κάθε πιθανή τιμή του χαρακτηριστικού και ανάλογα με την τρέχουσα τιμή του, μια από αυτές της μεταβλητές είναι 1, ενώ όλες οι άλλες παραμένουν 0. Για παράδειγμα, αν είχαμε κάποια μεταβλητή Weekday η οποία περιέγραφε τη μέρα που έγινε μια πτήση, θα είχαμε 7 μεταβλητές (π.χ. 1000000 για Monday, 0100000 για Tuesday κτλ.). Σημειώστε ότι με την one-hot encoding αναπαράσταση ο αριθμός των χαρακτηριστικών που προκύπτει είναι ίσος με τον αριθμό των διαφορετικών τιμών που παίρνει η μεταβλητή. Παρακάτω εφαρμόζουμε one-hot encoding στις μεταβλητές `Departure` και `Arrival`.

In [13]:
from sklearn.preprocessing import OneHotEncoder
cf = [0,1,4]
enc = OneHotEncoder(categorical_features=cf,sparse=False)#categorical_features=cf,sparse=False
enc.fit(df_train)
#enc.fit(df_test)
X_train = enc.transform(df_train)
X_test = enc.transform(df_test)
X_train.shape

C:\Users\IAKOVOS\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\IAKOVOS\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


(8899, 595)

In [14]:
print(X_train.shape[0])
print(X_train.shape[1])

8899
595


In [15]:
#PCA
'''
from sklearn.decomposition import PCA
samples = X_train.shape[0]
dim = X_train.shape[1]
n_components=1
pca = PCA()
Xtr_reduced = pca.fit_transform(X_train)
Xte_reduced = pca.fit_transform(X_test)
explained_variance = pca.explained_variance_ratio_  
print(explained_variance)
#Xtr_reduced2 = pca.fit_transform(X_train)
#Xte_reduced2 = pca.fit_transform(X_test)
'''

'\nfrom sklearn.decomposition import PCA\nsamples = X_train.shape[0]\ndim = X_train.shape[1]\nn_components=1\npca = PCA()\nXtr_reduced = pca.fit_transform(X_train)\nXte_reduced = pca.fit_transform(X_test)\nexplained_variance = pca.explained_variance_ratio_  \nprint(explained_variance)\n#Xtr_reduced2 = pca.fit_transform(X_train)\n#Xte_reduced2 = pca.fit_transform(X_test)\n'

In [16]:
'''
def get_optimal_number_of_components():
    c=0
    j=0
    while(c<0.9):
        c=explained_variance[j]+c
        j=j+1
    return j
com=get_optimal_number_of_components()
print(com)
print(explained_variance[0])
pca = PCA(n_components=com)  
X_train = pca.fit_transform(X_train)  
X_test = pca.transform(X_test)  
'''

'\ndef get_optimal_number_of_components():\n    c=0\n    j=0\n    while(c<0.9):\n        c=explained_variance[j]+c\n        j=j+1\n    return j\ncom=get_optimal_number_of_components()\nprint(com)\nprint(explained_variance[0])\npca = PCA(n_components=com)  \nX_train = pca.fit_transform(X_train)  \nX_test = pca.transform(X_test)  \n'

In [29]:
'''
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.metrics import f1_score
y_train = np.ravel(y_train)
kfold = 5
skf = StratifiedKFold(n_splits=kfold, random_state=42)
for i, (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
    print('[Fold %d/%d]' % (i + 1, kfold))
    X, X_valid = X_train[train_index][:], X_train[test_index][:]
    y, y_valid = y_train[train_index], y_train[test_index]
    clf = LogisticRegression()
    clf.fit(X, y)
    y_pred = clf.predict(X_valid)
    score=f1_score(y_valid, y_pred, average='micro')
    print('Fold %d/%d Prediciton: ' % (i + 1, kfold), score)
'''

[Fold 1/5]


C:\Users\IAKOVOS\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\IAKOVOS\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Fold 1/5 Prediciton:  0.3423817863397548
[Fold 2/5]


C:\Users\IAKOVOS\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\IAKOVOS\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Fold 2/5 Prediciton:  0.3649122807017544
[Fold 3/5]


...

Fold 3/5 Prediciton:  0.3992963940193492
[Fold 4/5]


C:\Users\IAKOVOS\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\IAKOVOS\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Fold 4/5 Prediciton:  0.3829225352112676
[Fold 5/5]


C:\Users\IAKOVOS\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\IAKOVOS\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Fold 5/5 Prediciton:  0.3783068783068783


Βλέπουμε ότι ο αριθμός των στηλών αυξήθηκε από 2 σε 40. Αυτό συνέβη γιατί υπάρχουν 20 διαφορετικά αεροδρόμια και συνεπώς χρειαζόμαστε 20 μεταβλητές για να αναπαραστήσουμε κάθε μια από τις μεταβλητές `Departure` και `Arrival` χρησιμοποιώντας one-hot encoding.

Έπειτα, εκπαιδεύουμε ξανά έναν ταξινομητή logistic regression για να προβλέψουμε τις κατηγορίες αριθμού επιβατών των δεδομένων ελέγχου. Επιπλέον, αποθηκεύουμε τις προβλέψεις μας στο αρχείο `y_pred.csv` στο δίσκο.

In [26]:
from sklearn.linear_model import LogisticRegression
import numpy as np
y_train = np.ravel(y_train)
#clf=LogisticRegression(C=0.5,solver='lbfgs', max_iter=200, multi_class='multinomial', verbose=1)
clf = LogisticRegression()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("Coefficients (b1 and b2) of the eight logistic regression models used in One vs Rest (ovr) scheme:\n "+ str(clf.coef_))
print("Intercept (b0) of the eight logistic regression models used in One vs Rest (ovr) scheme:\n "+ str(clf.intercept_))
#print (clf.coef_ + clf.intercept_)

C:\Users\IAKOVOS\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\IAKOVOS\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Coefficients (b1 and b2) of the eight logistic regression models used in One vs Rest (ovr) scheme:
 [[-5.13893402e-01 -1.64969005e-01  1.58114407e-02 ... -3.48300126e-02
  -1.00229648e-02 -6.77989812e-03]
 [-3.37069774e-01 -1.11221882e-01 -3.94118888e-01 ...  5.59884982e-04
   2.53750113e-02 -2.34401358e-02]
 [-4.43420273e-01 -3.67138259e-01  1.57984334e+00 ...  1.72552600e-02
   1.44045543e-02 -1.06663069e-01]
 ...
 [-7.68448366e-01 -8.37201197e-01  1.25799869e+00 ... -2.57542542e-02
   3.86282444e-03  9.02830070e-03]
 [ 8.17470148e-01  6.94905749e-01 -2.69184916e-01 ... -1.28886710e-02
   1.19856994e-05 -1.68334988e-03]
 [ 6.34009912e-01 -3.51274968e-01 -7.17725973e-01 ...  3.16738783e-03
   2.48527329e-02  4.19798024e-02]]
Intercept (b0) of the eight logistic regression models used in One vs Rest (ovr) scheme:
 [ 0.22356953 -1.19089231 -0.11396632 -0.35427525 -0.03938063  0.08513156
 -0.19114657 -0.38742226]


In [ ]:
#I MUST FIX IT
# Plot the decision boundary
x_min, x_max = X[:, 0].min() - 5, X[:, 0].max() + 5
y_min, y_max = X[:, 1].min() - 5, X[:, 1].max() + 5

plot_x = np.array([x_min, x_max])
plot_y = (- 1.0 / logreg.coef_[0,1]) * (logreg.coef_[0,0] * plot_x + logreg.intercept_)
plt.plot(plot_x, plot_y)
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=custom_cmap)
plt.xlabel('Exam 1 score')
plt.ylabel('Exam 2 score')

plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)

In [27]:
import csv
with open('y_predlr.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['Id', 'Label'])
    for i in range(y_pred.shape[0]):
        writer.writerow([i, y_pred[i]])

In [ ]:
#f1_score(y_test, y_pred, average='micro')

Υποβάλλουμε το αρχείο `y_pred.csv` στην πλατφόρμα και μας δίνει macro F1-score ίσο με 0.33. Στα πλαίσια της παρούσας εργασίας καλείστε να τροποποιήσετε τον παραπάνω κώδικα ώστε να προβλέψετε τις κατηγορίες αριθμού επιβατών των πτήσεων του test set. Μπορείτε να εφαρμόσετε κάποια μέθοδο επιλογής χαρακτηριστικών στα δεδομένα ώστε να κρατήσετε
μόνο ένα υποσύνολο από τα χαρακτηριστικά. Μπορείτε επίσης να δημιουργήσετε νέα χαρακτηριστικά τα οποία
ίσως βοηθήσουν στην κατηγοριοποίηση. Μπορείτε επιπλέον να πειραματιστείτε με κάποια μέθοδο μείωσης
διάστασης και να διερευνήσετε αν η εφαρμογή της βελτιώνει το αποτέλεσμα της κατηγοριοποίησης. Επίσης,
μπορείτε να χρησιμοποιήσετε θορυβώδη ή ανούσια χαρακτηριστικά για να παράγετε νέα χαρακτηριστικά που
παρέχουν μεγαλύτερα ποσοστά πληροφορίας. Μπορείτε να χρησιμοποιήσετε διαφορετικούς ταξινομητές ή να συνδυάσετε τα αποτελέσματα περισσότερων από έναν ταξινομητές. 

### Παράδοση Εργασίας

Η εργασία είναι είτε ατομική ή μπορεί να γίνει σε ομάδες το πολύ 3 ατόμων. Η τελική αξιολόγηση θα βασίζεται τόσο στο micro F1-score που θα επιτύχετε, όσο και στη συνολική προσέγγισή σας στο πρόβλημα. Στα πλαίσια της εργασίας, θα πρέπει να υποβληθούν τα εξής:
<ul>
    <li>Μια αναφορά 2 σελίδων, στην οποία θα πρέπει να περιγράψετε την προσέγγιση και τις μεθόδους που χρησιμοποιήσατε. Δεδομένου ότι πρόκειται για ένα πραγματικό πρόβλημα ταξινόμησης, μας ενδιαφέρει να γνωρίζουμε πώς αντιμετωπίσατε κάθε στάδιο του προβλήματος, π.χ. τι είδους αναπαράσταση δεδομένων χρησιμοποιήσατε, τι χαρακτηριστικά χρησιμοποιήσατε, εάν εφαρμόσατε τεχνικές μείωσης διάστασης, ποιούς αλγορίθμους ταξινόμησης δοκιμάσατε και γιατί, την απόδοση των μεθόδων σας (macro F1-score και χρόνο εκπαίδευσης), τυχόν προσεγγίσεις που τελικά δεν λειτούργησαν, αλλά
είναι ενδιαφέρον να παρουσιάσετε, και γενικά, ότι νομίζετε ότι είναι ενδιαφέρον να αναφερθεί.</li>
    <li>Ενα φάκελο με τον κώδικα της εφαρμογής σας.</li>
    <li>Εναλλακτικά μπορείτε να συνδυάσετε τα δυο παραπάνω σε ένα αρχείο Ipython Notebook.</li>
    <li>Δημιουργήστε ένα αρχείο .zip που περιέχει τον κώδικα και την αναφορά και υποβάλετέ τον στην πλατφόρμα e-class.</li>
    <li>**Λήξη προθεσμίας**: 6 Ιανουαρίου 2019.</li>
</ul>